In [7]:
import os, numpy, PIL
from PIL import Image
import datetime
import time
import cv2
import os
import traceback
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

In [8]:
def get_date_taken(path):

    try:
        datetime_str = Image.open(path)._getexif()[36867]
        datetime_obj = datetime.datetime.strptime(datetime_str, '%Y:%m:%d %H:%M:%S')
    except:
        return datetime.datetime.now()

    return datetime_obj

In [9]:
def create_video (ip_dir, op_dir, res, frame_rate=24, every_nth_image=1, ext='.jpg', sort_date=False):
    
    tic = datetime.datetime.now()

    splt = ip_dir.split('/')
    sub_dir = splt[len(splt)-1]
    
    images = [img for img in os.listdir(ip_dir) if not img.startswith('.') and img.lower().endswith(ext)]
    modified_dates = []
    date_available_flag = True
    
    for image in images:
        # image = os.path.join(ip_dir, image)
        mod_date = get_date_taken(image)
        if (mod_date == None):
            date_available_flag = False
            break
        modified_dates.append(mod_date)

    print('Image count =', len(images), end=' : ')
    if (len(images) == 0):
        print('No images found with extension ' + ext)
        return
    
    df_data = pd.DataFrame(list(zip(images, modified_dates)), columns=['filename', 'image_date'])
       
    if (sort_date):
        df_data = df_data.sort_values(by=['image_date'], ascending=True)
    else:
        df_data = df_data.sort_values(by=['filename'], ascending=True)
        
    df_data['filename'] = df_data['filename'].apply(lambda x: os.path.join(ip_dir, x))

    images = df_data['filename']
    
    if res is None:
        frame = cv2.imread(os.path.join(ip_dir, images[0]))
        height, width, layers = frame.shape
        res = (width, height)
    else:
        width, height = res
        
    print('Height:',height,'Width:', width)
    
    op_file = os.path.join(op_dir, sub_dir+'_'+str(frame_rate)+'fps_'+str(width)+'x'+str(height)+'.avi')
    print('Output Video: ', op_file)
    video = cv2.VideoWriter(op_file, 0, frame_rate, res)

    temp_file = 'temp.jpg'
    
    count = 0
    for image in images:

        try:
            if (count%every_nth_image == 0):
                image = os.path.join(ip_dir, image)

                im = Image.open(image)
                im_resized = im.resize(res, Image.ANTIALIAS)
                im_resized.save(temp_file)

                video.write(cv2.imread(temp_file))
                # print(image)

            count += 1
            print('Frames ('+str(count)+'/'+str(len(images))+') Elapsed Time:',str(datetime.datetime.now()-tic), end='\r')
        except:
            traceback.print_exc()
            continue

    if os.path.exists(temp_file):
        os.remove(temp_file)

    cv2.destroyAllWindows()
    video.release()

In [10]:
INPUT_DIR = '/Volumes/Mac Hawaii/Photos/Timelapse_'
OUTPUT_DIR = '/Users/talat/Pictures/_Photos/Video/'

INPUT_DIR = '/Users/talat/Pictures/_Photos/Edited/'
OUTPUT_DIR = '/Users/talat/Pictures/_Photos/Video/'

INPUT_DIR = '/Volumes/PhotosAug20/Photos/Edited/'
OUTPUT_DIR = '/Volumes/PhotosAug20/Photos/Video/'

INPUT_DIR = '/Users/talat/Pictures/_Photos/Edited/'


# INPUT_DIR = '/Volumes/TOSHIBA/Photos/TL Edited/'
# OUTPUT_DIR = '/Volumes/TOSHIBA/Photos/Videos/'

# INPUT_DIR = '/Users/talat/Pictures/_Photos/TL Edited1/'
OUTPUT_DIR = '/Users/talat/Pictures/_Photos/Video/'
# OUTPUT_DIR = '/Volumes/OG 2/TL Videos Uncompressed/'

OUTPUT_DIR = '/Volumes/Empty/TL Videos Uncompressed 2/'
# OUTPUT_DIR = '/Volumes/Empty/Timelapse Videos Uncompressed - 2/' 
# OUTPUT_DIR = '/Users/talat/Pictures/_Photos/Timelapse Videos Uncompressed/'

res_hd = (1920, 1080)
res_4k = (3840, 2160)
res = res_hd
res = None

every_nth_image = 1
frame_rate = 24
ext = '.jpg'

# Create output directories
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

sub_dir_list = [f for f in os.listdir(INPUT_DIR) if os.path.isdir(os.path.join(INPUT_DIR,f))]
sub_dir_list.sort()

print('Number of sub-directories found: '+str(len(sub_dir_list)))
count = 1

for sub_dir in sub_dir_list:
    
    try:
        tic = datetime.datetime.now()
        ip_dir = os.path.join(INPUT_DIR, sub_dir)
        op_dir = OUTPUT_DIR

        print('['+str(count)+'/'+str(len(sub_dir_list))+'] '+sub_dir, end=' : ')

        create_video(ip_dir=ip_dir, op_dir=op_dir, res=res, frame_rate=frame_rate, 
                     every_nth_image=every_nth_image, ext=ext)
        print('Video created in ', str(datetime.datetime.now()-tic), 'seconds')
    except:
        print('EXCEPTION: having issues with the directory: '+sub_dir)
        traceback.print_exc()
        continue
    
    count += 1

Number of sub-directories found: 4
[1/4] 2021-06-14 - Icefield Parkway Peak (TL) 1MB WM RT E2 2 : Image count = 189 : Height: 4000 Width: 6000
Output Video:  /Volumes/Empty/TL Videos Uncompressed 2/2021-06-14 - Icefield Parkway Peak (TL) 1MB WM RT E2 2_24fps_6000x4000.avi
Video created in  0:05:37.557108 seconds67816
[2/4] 2021-06-14 - Icefield Parkway Peak (TL) 1MB WM RT E2 LE : Image count = 180 : Height: 4000 Width: 6000
Output Video:  /Volumes/Empty/TL Videos Uncompressed 2/2021-06-14 - Icefield Parkway Peak (TL) 1MB WM RT E2 LE_24fps_6000x4000.avi
Video created in  0:05:33.450979 seconds12354
[3/4] 2021-06-29 - Dahlia Flowering Bud - Run 3 1in3 1MB E2 : Image count = 1216 : Height: 4000 Width: 3793
Output Video:  /Volumes/Empty/TL Videos Uncompressed 2/2021-06-29 - Dahlia Flowering Bud - Run 3 1in3 1MB E2_24fps_3793x4000.avi
Video created in  0:19:49.464147 seconds.741740
[4/4] 2021-06-29 - Dahlia Flowering Bud - Run 3 1in3 1MB WM RB E3 : Image count = 1216 : Height: 4000 Width: 3